# Qwen3-32B LoRA 微调完整教程

本 Notebook 展示如何使用 XPULink API 对 Qwen3-32B 进行 LoRA (Low-Rank Adaptation) 微调。

## 什么是 LoRA?

LoRA (Low-Rank Adaptation) 是一种参数高效的微调方法:
- ✅ **高效**: 只训练少量参数,显著降低计算成本
- ✅ **快速**: 训练速度快,收敛迅速
- ✅ **灵活**: 可以为不同任务训练多个 LoRA 适配器
- ✅ **经济**: 相比全参数微调,成本大幅降低

## 使用场景

- 特定领域知识注入 (医疗、法律、金融等)
- 企业内部知识库定制
- 特定风格的文本生成
- 专业领域的对话系统

## 步骤 1: 环境准备

安装必要的依赖包

In [ ]:
# 安装依赖
!pip install requests python-dotenv -q

In [ ]:
# 导入必要的库
import os
import json
import requests
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 获取 API Key
API_KEY = os.getenv("XPULINK_API_KEY")

if not API_KEY:
    print("⚠️  警告: 未找到 XPULINK_API_KEY 环境变量")
    print("请在项目根目录的 .env 文件中设置: XPULINK_API_KEY=your_api_key")
else:
    print("✅ API Key 已加载")

## 步骤 2: 准备训练数据

训练数据格式为 JSONL (每行一个 JSON 对象),每个对象包含一个完整的对话。

In [ ]:
# 示例: 创建Python编程助手的训练数据
training_data = [
    {
        "messages": [
            {"role": "system", "content": "你是一个专业的Python编程助手,擅长解释Python概念和提供代码示例。"},
            {"role": "user", "content": "什么是Python装饰器?"},
            {"role": "assistant", "content": "装饰器是Python中一种强大的设计模式,用于在不修改原函数代码的情况下增加功能。\n\n基本示例:\n```python\ndef my_decorator(func):\n    def wrapper():\n        print('函数执行前')\n        func()\n        print('函数执行后')\n    return wrapper\n\n@my_decorator\ndef say_hello():\n    print('Hello!')\n```"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "你是一个专业的Python编程助手,擅长解释Python概念和提供代码示例。"},
            {"role": "user", "content": "如何使用Python读取CSV文件?"},
            {"role": "assistant", "content": "读取CSV文件可以使用pandas库,这是最常用的方法:\n\n```python\nimport pandas as pd\n\n# 读取CSV文件\ndf = pd.read_csv('data.csv')\n\n# 查看前几行\nprint(df.head())\n```\n\n如果不使用pandas,也可以用内置的csv模块:\n```python\nimport csv\n\nwith open('data.csv', 'r', encoding='utf-8') as f:\n    reader = csv.reader(f)\n    for row in reader:\n        print(row)\n```"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "你是一个专业的Python编程助手,擅长解释Python概念和提供代码示例。"},
            {"role": "user", "content": "Python中如何处理异常?"},
            {"role": "assistant", "content": "Python使用try-except语句处理异常:\n\n```python\ntry:\n    # 可能出错的代码\n    result = 10 / 0\nexcept ZeroDivisionError:\n    print('除数不能为0')\nexcept Exception as e:\n    print(f'发生错误: {e}')\nfinally:\n    print('无论是否出错都会执行')\n```\n\n最佳实践:\n1. 捕获具体的异常类型\n2. 避免使用空except\n3. 在finally中清理资源"}
        ]
    },
    # 添加更多训练样本以提高效果...
    # 建议至少准备 50-100 个高质量样本
]

# 保存为 JSONL 格式
os.makedirs('data', exist_ok=True)
training_file = 'data/training_data.jsonl'

with open(training_file, 'w', encoding='utf-8') as f:
    for item in training_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✅ 训练数据已保存到: {training_file}")
print(f"📊 训练样本数: {len(training_data)}")
print("\n💡 提示: 实际应用中建议准备至少 50-100 个高质量训练样本以获得更好的效果")

## 步骤 3: 上传训练文件

将准备好的训练数据上传到 XPULink 平台

In [ ]:
def upload_training_file(file_path, api_key):
    """上传训练文件到 XPULink"""
    url = "https://www.xpulink.ai/v1/files"
    
    with open(file_path, 'rb') as f:
        files = {
            'file': (os.path.basename(file_path), f, 'application/json'),
            'purpose': (None, 'fine-tune')
        }
        
        headers = {"Authorization": f"Bearer {api_key}"}
        
        response = requests.post(url, headers=headers, files=files, timeout=60)
        
        if response.status_code != 200:
            raise Exception(f"上传失败: {response.text}")
        
        result = response.json()
        return result.get('id')

# 上传文件
print("📤 正在上传训练文件...")
file_id = upload_training_file(training_file, API_KEY)
print(f"✅ 文件上传成功!")
print(f"📄 文件 ID: {file_id}")

## 步骤 4: 创建 LoRA 微调任务

配置超参数并创建微调任务

In [ ]:
def create_finetune_job(file_id, api_key, model="qwen3-32b", suffix=None):
    """创建微调任务"""
    url = "https://www.xpulink.ai/v1/fine_tuning/jobs"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    # LoRA 超参数配置
    payload = {
        "training_file": file_id,
        "model": model,
        "hyperparameters": {
            "n_epochs": 3,              # 训练轮数 (可调整 1-10)
            "batch_size": 4,            # 批次大小 (根据数据量调整)
            "learning_rate": 5e-5,      # 学习率 (典型值: 1e-5 到 1e-4)
            "lora_r": 8,                # LoRA 秩 (典型值: 4, 8, 16)
            "lora_alpha": 16,           # LoRA alpha (通常是 lora_r 的 2 倍)
            "lora_dropout": 0.05        # LoRA dropout (0.0 - 0.1)
        }
    }
    
    if suffix:
        payload["suffix"] = suffix
    
    response = requests.post(url, headers=headers, json=payload, timeout=30)
    
    if response.status_code not in [200, 201]:
        raise Exception(f"创建任务失败: {response.text}")
    
    return response.json()

# 创建微调任务
print("🚀 创建微调任务...")
job = create_finetune_job(
    file_id=file_id,
    api_key=API_KEY,
    model="qwen3-32b",
    suffix="python-assistant"  # 可选: 为微调模型添加后缀
)

job_id = job.get('id')
print(f"✅ 微调任务创建成功!")
print(f"🆔 任务 ID: {job_id}")
print(f"\n📊 任务详情:")
print(json.dumps(job, indent=2, ensure_ascii=False))

## 步骤 5: 监控微调进度

检查微调任务状态

In [ ]:
import time

def check_job_status(job_id, api_key):
    """检查微调任务状态"""
    url = f"https://www.xpulink.ai/v1/fine_tuning/jobs/{job_id}"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    response = requests.get(url, headers=headers, timeout=30)
    
    if response.status_code != 200:
        raise Exception(f"获取状态失败: {response.text}")
    
    return response.json()

# 检查任务状态
print("📊 检查任务状态...\n")
status = check_job_status(job_id, API_KEY)

current_status = status.get('status')
print(f"当前状态: {current_status}")

if current_status == 'succeeded':
    print(f"✅ 微调完成!")
    print(f"🎉 微调模型: {status.get('fine_tuned_model')}")
elif current_status in ['running', 'pending', 'queued']:
    print(f"⏳ 任务进行中,请稍后再次检查")
    print(f"💡 提示: 微调通常需要几分钟到几小时,具体取决于数据量和模型大小")
elif current_status == 'failed':
    print(f"❌ 任务失败: {status.get('error')}")

print(f"\n完整状态信息:")
print(json.dumps(status, indent=2, ensure_ascii=False))

## 步骤 6: 使用微调后的模型

微调完成后,您可以像使用基础模型一样调用微调模型

In [ ]:
def test_finetuned_model(model_name, prompt, api_key, max_tokens=300):
    """测试微调模型"""
    url = "https://www.xpulink.ai/v1/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": 0.7
    }
    
    response = requests.post(url, headers=headers, json=payload, timeout=60)
    
    if response.status_code != 200:
        raise Exception(f"调用失败: {response.text}")
    
    result = response.json()
    return result["choices"][0]["message"]["content"]

# 获取微调模型名称 (从上一步的状态中获取)
finetuned_model = status.get('fine_tuned_model')

if finetuned_model:
    print(f"🧪 测试微调模型: {finetuned_model}\n")
    
    # 测试问题
    test_prompts = [
        "什么是Python生成器?",
        "如何在Python中实现多线程?",
        "解释一下Python中的上下文管理器"
    ]
    
    for i, prompt in enumerate(test_prompts, 1):
        print(f"\n{'='*60}")
        print(f"测试 {i}: {prompt}")
        print(f"{'='*60}")
        
        response = test_finetuned_model(finetuned_model, prompt, API_KEY)
        print(f"\n🤖 模型回答:\n{response}")
else:
    print("⚠️  微调尚未完成,无法测试模型")
    print("请等待微调完成后再运行此单元格")

## 步骤 7: 列出所有微调任务 (可选)

查看您账号下的所有微调任务

In [ ]:
def list_finetune_jobs(api_key, limit=10):
    """列出微调任务"""
    url = f"https://www.xpulink.ai/v1/fine_tuning/jobs?limit={limit}"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    response = requests.get(url, headers=headers, timeout=30)
    
    if response.status_code != 200:
        raise Exception(f"获取列表失败: {response.text}")
    
    return response.json().get('data', [])

# 列出任务
print("📋 您的微调任务列表:\n")
jobs = list_finetune_jobs(API_KEY, limit=10)

if not jobs:
    print("📭 暂无微调任务")
else:
    for i, job in enumerate(jobs, 1):
        print(f"{i}. 任务 ID: {job.get('id')}")
        print(f"   状态: {job.get('status')}")
        print(f"   基础模型: {job.get('model')}")
        print(f"   创建时间: {job.get('created_at')}")
        if job.get('fine_tuned_model'):
            print(f"   微调模型: {job.get('fine_tuned_model')}")
        print()

## 总结和最佳实践

### 数据准备建议
1. **数据量**: 至少准备 50-100 个高质量训练样本
2. **数据质量**: 确保对话格式正确,回答准确且一致
3. **数据多样性**: 覆盖目标任务的各种场景
4. **格式规范**: 严格遵循 JSONL 格式要求

### 超参数调优建议
- **n_epochs**: 3-5 轮通常效果较好,过多可能过拟合
- **learning_rate**: 从 5e-5 开始尝试,可在 1e-5 到 1e-4 之间调整
- **lora_r**: 较小的值 (4-8) 参数更少但可能效果稍差,较大的值 (16-32) 效果更好但参数更多
- **batch_size**: 根据数据量和 API 限制调整

### 成本优化
- 从小规模数据开始测试
- 使用验证集评估模型效果
- 避免不必要的重复训练

### 注意事项
- ⚠️ 本示例基于 OpenAI Fine-tuning API 标准,实际 XPULink API 可能有所不同
- ⚠️ 请参考 XPULink 官方文档了解最新的 API 接口规范
- ⚠️ 微调需要一定时间,请耐心等待任务完成
- ⚠️ 妥善保管 API Key,不要泄露到公开代码中